In [1]:
#!pip3 install librosa
#!pip3 install sounddevice
#!pip3 install pyaudio  #there will be gcc error
#installing this before installing pyaudio worked for me
#sudo apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0 
import IPython.display as ipd
from ipywidgets import widgets
from functools import partial
%matplotlib inline
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pyaudio
import numpy as np
import time

In [ ]:
audio_path = './Icon.wav'
x , sr = librosa.load(audio_path)
print(type(x), type(sr))
print(x.shape, sr)


plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

librosa.output.write_wav('example.wav', x, sr)



In [ ]:
ipd.display(ipd.Audio(data=x, rate = sr)) #  if not wrap in ipd.display, then timesize will be uncorrect, and slider doesn't work
#ipd.display(ipd.Audio(filename=audio_path))

In [ ]:
#Частота цветности (chroma features)
hop_length = 512
#time_stop = 1000000
#showed_x = x[0:time_stop]
showed_x = x
chromagram = librosa.feature.chroma_stft(showed_x, sr=sr, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')

In [ ]:
# using sounddevice to get audiostream
duration = 10.5  # seconds
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=2)
# stop recording
sd.wait()


In [ ]:
# 03.04 try to using pyaudio for realtime
# find funny idea: https://www.youtube.com/watch?v=at2NppqIZok
#import pyaudio


RATE    = 16000
CHUNK   = 256

p = pyaudio.PyAudio()

player = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, output=True, 
frames_per_buffer=CHUNK)
stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNK)

for i in range(int(20*RATE/CHUNK)): #do this for 10 seconds
    player.write(np.fromstring(stream.read(CHUNK),dtype=np.int16))
stream.stop_stream()
stream.close()
p.terminate()



In [ ]:

CHANNELS = 2
RATE = 44100
CHUNK   = 1024 #1024 by default
p = pyaudio.PyAudio()

def callback(in_data, frame_count, time_info, flag):
    # using Numpy to convert to array for processing
    audio_data = np.frombuffer(in_data, dtype=np.float32)
    return audio_data, pyaudio.paContinue
    #return in_data, pyaudio.paContinue

stream = p.open(format=pyaudio.paFloat32,
                channels=CHANNELS,
                rate=RATE,
                output=True,
                input=True,
                stream_callback=callback,frames_per_buffer=CHUNK)

stream.start_stream()
print("Stream is started")
# TODO: add widget to manually start and stop recording
while stream.is_active(): 
    time.sleep(10)
# if you don't call stream.stop_stream() it will continue to work even after cell finished
#that's funny, but don't do that, (restarting kernel helps to stop recording)
    stream.stop_stream()
    print("Stream is stopped")

stream.close()

p.terminate()

In [2]:
#define audio settings and data processing

CHANNELS = 2
RATE = 44100
CHUNK   = 1024

def callback(in_data, frame_count, time_info, flag):
    # using Numpy to convert to array for processing
    audio_data = np.frombuffer(in_data, dtype=np.float32)
    #np.multiply(audio_data,100)
    return audio_data, pyaudio.paContinue
    #return in_data, pyaudio.paContinue
    

In [3]:
# define widget

# here your button layout, basically it's size and border
layout = widgets.Layout(
    width='30px', height='60px',
    border='1px solid black')

# don't check why, but you need to add an extra parameter in signature of method (there it's 'b'), or it won't work
def on_button_start_clicked(stream,b):
    stream.start_stream()
    print("Stream is started")
    
def on_button_stop_clicked(stream,b):
    stream.stop_stream()
    print("Stream is stopped")

In [4]:
# create widget for easy working with speech recorder
#define our audio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32,
                channels=CHANNELS,
                rate=RATE,
                output=True,
                input=True,
                stream_callback=callback,frames_per_buffer=CHUNK)

# here your buttons
buttons = []
button_start = widgets.Button(
        description='start', layout=layout)
button_stop = widgets.Button(
        description='pause', layout=layout)
button_start.on_click(partial(on_button_start_clicked,stream))
#button_stop.on_click(partial(on_button_stop_clicked,stream))
button_stop.on_click(partial(on_button_stop_clicked,stream))

buttons.append(button_start)
buttons.append(button_stop)

# here your buttons in horizontal line
widgets.Box(children=buttons)


Box(children=(Button(description='start', layout=Layout(border='1px solid black', height='60px', width='30px')…

In [ ]:
#Call this cell for emergency
stream.close()
p.terminate()